# XLMRat PCAP Malware Analysis Notebook

## 📥 Step 1: Load the PCAP File
First, we'll use **Scapy** to load and parse the PCAP file, which contains network packets captured during a potential malware infection.

In [13]:
# Import necessary Scapy components
from scapy.all import rdpcap, TCP, Raw, IP

# Load the PCAP file
packets = rdpcap('236-XLMRat.pcap')
print(f"Total packets loaded: {len(packets)}")

Total packets loaded: 1550


## 🌐 Step 2: Extract HTTP Requests
We filter packets for HTTP GET or POST requests to identify URLs accessed by the infected machine. This helps pinpoint where the malware was downloaded from.

In [14]:
# Extract and reconstruct HTTP requests from TCP payloads
http_requests = []
for pkt in packets:
    if pkt.haslayer(TCP) and pkt.haslayer(Raw):
        try:
            payload = pkt[Raw].load.decode('utf-8', errors='ignore')
            if 'GET' in payload or 'POST' in payload:
                lines = payload.splitlines()
                method = lines[0] if lines else ''
                host = next((line.split(':', 1)[1].strip() for line in lines if line.lower().startswith('host:')), '')
                uri = method.split(' ')[1] if ' ' in method else ''
                full_url = f"http://{host}{uri}"
                http_requests.append(full_url)
        except Exception:
            continue

print("🔗 URLs extracted from the PCAP:")
for url in http_requests:
    print(url)

🔗 URLs extracted from the PCAP:
http://45.126.209.4:222/xlm.txt
http://45.126.209.4:222/mdm.jpg


## 🛰️ Step 3: Determine Hosting Provider
Now we take the host from the malicious URL and determine the IP address and hosting provider using `socket` and `requests`.

In [15]:
import socket
import requests
from urllib.parse import urlparse

# Parse the IP address from the first suspicious URL
target_url = http_requests[0]
host = urlparse(target_url).hostname
ip = socket.gethostbyname(host)

# Query ipinfo.io to get hosting provider
resp = requests.get(f"https://ipinfo.io/{ip}/org")
print(f"IP Address: {ip}")
print(f"Hosting Provider: {resp.text.strip()}")

IP Address: 45.126.209.4
Hosting Provider: AS23470 ReliableSite.Net LLC
